In [1]:
import pandas as pd

In [2]:
raw_data = pd.read_csv('./data/Seeds/data_sequence.txt', sep=' ')

In [3]:
# raw_data

In [4]:
# go throught the "species" column and strip spaces " " from the beginning and end of the string
raw_data['species'] = raw_data['species'].str.strip()

In [5]:
raw_data.head().to_csv("sample_sequence.csv")

In [6]:
# defiine kmerfunction
def Kmers_funct(seq, size):
    return [seq[x:x+size].lower() for x in range(len(seq) - size + 1)]

In [7]:
# # please apply the function to the "sequence" column and save the result in a new column called "joined_sequence"
# raw_data['joined_sequence'] = raw_data['sequence'].apply(lambda x: ' '.join(Kmers_funct(x, size=4)))


In [8]:
# raw_data

In [9]:
# import pandas as pd

# #Creating the Bag of Words model:
# from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer()
# X = cv.fit_transform(raw_data['joined_sequence']).toarray()

# # Get feature names
# feature_names = cv.get_feature_names_out()

# # Create a DataFrame from X with feature names as columns
# df = pd.DataFrame(X, columns=feature_names)

# # add the species as index to the DataFrame
# df.index = raw_data['species']

In [10]:
def generate_kmers_dataset(size):
    # Apply the Kmers_funct to the 'sequence' column
    raw_data['joined_sequence'] = raw_data['sequence'].apply(lambda x: ' '.join(Kmers_funct(x, size=size)))

    # Create the Bag of Words model
    from sklearn.feature_extraction.text import CountVectorizer
    cv = CountVectorizer()
    X = cv.fit_transform(raw_data['joined_sequence']).toarray()

    # Get feature names
    feature_names = cv.get_feature_names_out()

    # Create a DataFrame from X with feature names as columns
    df = pd.DataFrame(X, columns=feature_names)

    # Add the 'species' as index to the DataFrame
    df.index = raw_data['species']

    # Merge the DataFrame with y
    y = pd.read_parquet('relevant_micros_avg_abundance.gz')
    df = df.merge(y, left_index=True, right_index=True)

    # # Save the DataFrame to a parquet file
    # df.to_parquet(f'seq_{size}mers_input_data.gz')

    return df

In [11]:
df = generate_kmers_dataset(4)

In [12]:
# for size in range(1, 10):
#     df = generate_kmers_dataset(size)

In [13]:
# df

## ML Part

In [14]:
# df

In [15]:
cols_to_drop = df.iloc[:,334:].columns.tolist()

In [16]:
len(cols_to_drop)

372

## Bsic RF

In [14]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Assume 'df' is already created with k-mer features and microorganism abundance
# Load 'df' if needed
# df = pd.read_parquet('path_to_your_parquet_file')

# Define the target microorganism abundance column
target_col = '4b2764a206cf4cab3937a52a9aca6079'

# Split the dataset into features (X) and target (y)
X = df.drop(columns=cols_to_drop)
y = df[target_col]

# Define the parameter grid for RandomForestRegressor
param_grid = {
    'n_estimators': [200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize RandomForestRegressor
rf = RandomForestRegressor(random_state=42)

# Initialize GridSearchCV with 10-fold cross-validation to find the best parameters
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X, y)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Use the best parameters for LOOCV
best_rf = RandomForestRegressor(**best_params, random_state=42)

# Perform leave-one-out cross-validation
errors = []
species_names = []
trues = []
preds = []
for species in X.index:
    X_train = X.drop(index=species)
    y_train = y.drop(index=species)
    X_test = X.loc[[species]]
    y_test = y.loc[[species]]

    best_rf.fit(X_train, y_train)
    y_pred = best_rf.predict(X_test)
    error = mean_squared_error(y_test, y_pred)
    errors.append(error)
    species_names.append(species)  # Store the species name
    trues.append(y_test.values)
    preds.append(y_pred)

# Combine species names with their corresponding errors
error_df = pd.DataFrame({
    'Species': species_names,
    'MSE': errors,
    'trues':trues,
    'predictions' : preds
    
})

# Compute the average test error
average_test_error = np.mean(errors)
print("Average Test set error (MSE): ", average_test_error)




Process LokyProcess-22:
Process LokyProcess-12:
Process LokyProcess-13:
Process LokyProcess-16:
Process LokyProcess-11:
Process LokyProcess-8:
Process LokyProcess-15:
Process LokyProcess-14:
Process LokyProcess-2:
Process LokyProcess-6:
Process LokyProcess-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/jannik/miniconda3/envs/istari_24/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py", line 463, in _process_worker
    r = call_item()
  File "/home/jannik/miniconda3/envs/istari_24/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py", line 291, in __call__
    return self.fn(*self.args, **self.kwargs)
  File "/home/jannik/miniconda3/envs/istari_24/lib/python3.9/site-packages/joblib/parallel.py", line 598, in __call__
    return [func(*args, **kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent 

KeyboardInterrupt: 

In [14]:
# error_df

## RF with re-schaling

In [17]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Assume 'df' is already created with k-mer features and microorganism abundance
# Load 'df' if needed
# df = pd.read_parquet('path_to_your_parquet_file')

# Define the target microorganism abundance column
target_col = '4b2764a206cf4cab3937a52a9aca6079'

# Apply log transformation to the target variable
df[target_col] = np.log1p(df[target_col])

# Split the dataset into features (X) and target (y)
X = df.drop(columns=cols_to_drop)
y = df[target_col]

# Define the parameter grid for RandomForestRegressor
param_grid = {
    'n_estimators': [200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize RandomForestRegressor
rf = RandomForestRegressor(random_state=42)

# Initialize GridSearchCV with 10-fold cross-validation to find the best parameters
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X, y)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Use the best parameters for LOOCV
best_rf = RandomForestRegressor(**best_params, random_state=42)

# Perform leave-one-out cross-validation
errors = []
species_names = []
trues = []
preds = []


for species in X.index:
    X_train = X.drop(index=species)
    y_train = y.drop(index=species)
    X_test = X.loc[[species]]
    y_test = y.loc[[species]]

    best_rf.fit(X_train, y_train)
    y_pred = best_rf.predict(X_test)
    # transform back
    y_pred = np.expm1(y_pred)  # Inverse transform the prediction
    y_test = np.expm1(y_test)  # Inverse transform the actual value
    error = mean_squared_error(y_test, y_pred)
    errors.append(error)
    species_names.append(species)  # Store the species name
    trues.append(y_test.values)
    preds.append(y_pred)

# Combine species names with their corresponding errors
error_df = pd.DataFrame({
    'Species': species_names,
    'MSE': errors,
    'trues':trues,
    'predictions' : preds
    
})

# Compute the average test error
average_test_error = np.mean(errors)
print("Average Test set error (MSE): ", average_test_error)




Best parameters found:  {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 300}
Average Test set error (MSE):  2.1826629895938234


In [ ]:
# re-shaling in RF was not very useful


# Try Log-Transform and Other Algo- XGBoost

In [12]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Assume 'df' is already created with k-mer features and microorganism abundance
# Load 'df' if needed
# df = pd.read_parquet('path_to_your_parquet_file')

# Define the target microorganism abundance column
target_col = '4b2764a206cf4cab3937a52a9aca6079'

# Apply log transformation to the target variable
df[target_col] = np.log1p(df[target_col])

# Split the dataset into features (X) and target (y)
X = df.drop(columns=[target_col])
y = df[target_col]

# Define the parameter grid for XGBRegressor
param_grid = {
    'n_estimators': [ 200, 300],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.7, 0.8, 0.9]
}

# Initialize XGBRegressor
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)

# Initialize GridSearchCV with 10-fold cross-validation to find the best parameters
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X, y)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Use the best parameters for LOOCV
best_xgb = XGBRegressor(**best_params, objective='reg:squarederror', random_state=42)

# Perform leave-one-out cross-validation
errors = []
species_names = []
preds = []
trues = []

for species in X.index:
    X_train = X.drop(index=species)
    y_train = y.drop(index=species)
    X_test = X.loc[[species]]
    y_test = y.loc[[species]]
    
    best_xgb.fit(X_train, y_train)
    y_pred = best_xgb.predict(X_test)
    y_pred = np.expm1(y_pred)  # Inverse transform the prediction
    y_test = np.expm1(y_test)  # Inverse transform the actual value
    error = mean_squared_error(y_test, y_pred)
    errors.append(error)
    species_names.append(species)  # Store the species name
    trues.append(y_test.values)
    preds.append(y_pred)

# Combine species names with their corresponding errors
error_df = pd.DataFrame({
    'Species': species_names,
    'MSE': errors,
    'trues':trues,
    'predictions' : preds
})

# Compute the average test error
average_test_error = np.mean(errors)
print("Average Test set error (MSE): ", average_test_error)




Best parameters found:  {'learning_rate': 0.3, 'max_depth': 3, 'n_estimators': 300, 'subsample': 0.8}
Average Test set error (MSE):  0.31459247760671555


- okay, so the error dropped immensly using XGBoost regression!!
- the only thing left to check is whether the re-scaling is the driving force or whether its just the XGBoost
- check: XBboost only (no resclaing) --> checked, seems combination is best

In [17]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Assume 'df' is already created with k-mer features and microorganism abundance
# Load 'df' if needed
# df = pd.read_parquet('path_to_your_parquet_file')

# Define the target microorganism abundance column
target_col = '4b2764a206cf4cab3937a52a9aca6079'


# Split the dataset into features (X) and target (y)
X = df.drop(columns=[target_col])
y = df[target_col]

# Define the parameter grid for XGBRegressor
param_grid = {
    'n_estimators': [ 200, 300],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.7, 0.8, 0.9]
}

# Initialize XGBRegressor
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)

# Initialize GridSearchCV with 10-fold cross-validation to find the best parameters
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X, y)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Use the best parameters for LOOCV
best_xgb = XGBRegressor(**best_params, objective='reg:squarederror', random_state=42)

# Perform leave-one-out cross-validation
errors = []
species_names = []
preds = []
trues = []

for species in X.index:
    X_train = X.drop(index=species)
    y_train = y.drop(index=species)
    X_test = X.loc[[species]]
    y_test = y.loc[[species]]
    
    best_xgb.fit(X_train, y_train)
    y_pred = best_xgb.predict(X_test)

    error = mean_squared_error(y_test, y_pred)
    errors.append(error)
    species_names.append(species)  # Store the species name
    trues.append(y_test.values)
    preds.append(y_pred)

# Combine species names with their corresponding errors
error_df = pd.DataFrame({
    'Species': species_names,
    'MSE': errors,
    'trues':trues,
    'predictions' : preds
})

# Compute the average test error
average_test_error = np.mean(errors)
print("Average Test set error (MSE): ", average_test_error)




Best parameters found:  {'learning_rate': 0.3, 'max_depth': 3, 'n_estimators': 300, 'subsample': 0.8}
Average Test set error (MSE):  0.5204648434930192


In [19]:
# error_df

## BUT: keep in mind, not totally unseen, the "hold-out" was seen in training so its not really unseen!
- need to test this...

- -then: if still prevails,
- test different k-mers, feature engeneering etc...